In [1]:
import pandas as pd

In [3]:
movies=pd.read_csv('dataset.csv')

In [5]:
movies.head(10)

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
5,667257,Impossible Things,"Family,Drama",es,"Matilde is a woman who, after the death of her...",14.358,2021-06-17,8.6,255
6,129,Spirited Away,"Animation,Family,Fantasy",ja,"A young girl, Chihiro, becomes trapped in a st...",92.056,2001-07-20,8.5,13093
7,730154,Your Eyes Tell,"Romance,Drama",ja,"A tragic accident lead to Kaori's blindness, b...",51.345,2020-10-23,8.5,339
8,372754,Dou kyu sei – Classmates,"Romance,Animation",ja,"Rihito Sajo, an honor student with a perfect s...",14.285,2016-02-20,8.5,239
9,372058,Your Name.,"Romance,Animation,Drama",ja,High schoolers Mitsuha and Taki are complete s...,158.270,2016-08-26,8.5,8895


In [6]:
movies.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [8]:
movies.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

Feature Selection Part

In [9]:
movies.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle

# Load the dataset
movies = pd.read_csv('dataset.csv')

# Handling missing values
movies['overview'] = movies['overview'].fillna('')
movies['genre'] = movies['genre'].fillna('Unknown')
movies['tags'] = movies['overview'] + ' ' + movies['genre']

# Convert release_date to datetime and extract year (optional use)
movies['year'] = pd.to_datetime(movies['release_date']).dt.year

# Normalize popularity and vote_average
scaler = MinMaxScaler()
movies[['popularity', 'vote_average']] = scaler.fit_transform(movies[['popularity', 'vote_average']])

# Vectorization of the tags
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['tags'].astype('U'))

# Combine the TF-IDF matrix with the scaled popularity and vote_average
# Convert the sparse tfidf_matrix to a dense format using np.asarray to concatenate with the pandas dataframe
tfidf_dense = np.asarray(tfidf_matrix.todense())
features = np.hstack((tfidf_dense, movies[['popularity', 'vote_average']].values))

# Calculate cosine similarity from the combined feature matrix
combined_similarity = cosine_similarity(features)

# Recommendation function
def recommend(movie_title, num_recommendations=5):
    if movie_title not in movies['title'].values:
        return f"No movie found with title {movie_title}"
    index = movies[movies['title'] == movie_title].index[0]
    similarities = combined_similarity[index]
    movie_indices = np.argsort(-similarities)[1:num_recommendations+1]
    return movies.iloc[movie_indices][['title', 'year', 'popularity', 'vote_average','genre']]

# Example usage
print(recommend('The Godfather'))

# Save objects
pickle.dump(movies, open('movies.pkl', 'wb'))
pickle.dump(combined_similarity, open('combined_similarity.pkl', 'wb'))


                      title  year  popularity  vote_average  \
4    The Godfather: Part II  1974    0.005476      0.975610   
214       The Best of Youth  2003    0.000585      0.853659   
37                     Hope  2013    0.001590      0.926829   
25        Life Is Beautiful  1997    0.005263      0.951220   
330                    Rope  1948    0.001156      0.829268   

                     genre  
4              Drama,Crime  
214  Drama,History,Romance  
37                   Drama  
25            Comedy,Drama  
330   Crime,Drama,Thriller  


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load the dataset
movies = pd.read_csv('dataset.csv')

# Handling missing values
movies['overview'] = movies['overview'].fillna('')
movies['genre'] = movies['genre'].fillna('Unknown')
movies['tags'] = movies['overview'] + ' ' + movies['genre']

# Convert release_date to datetime and extract year
movies['year'] = pd.to_datetime(movies['release_date']).dt.year

# Normalize popularity and vote_average
scaler = MinMaxScaler()
movies[['popularity', 'vote_average']] = scaler.fit_transform(movies[['popularity', 'vote_average']])

# Feature Engineering
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['tags'].astype('U')).todense()

# Prepare data for the autoencoder
feature_array = np.hstack([tfidf_matrix, movies[['popularity', 'vote_average']].values])

# Build the autoencoder model
input_dim = feature_array.shape[1]
encoding_dim = 256  # Size of the embedding

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
encoder = Model(input_layer, encoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(feature_array, feature_array,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_split=0.1)

# Generate the movie embeddings
movie_embeddings = encoder.predict(feature_array)

# Compute similarities between movie embeddings
movie_similarity = cosine_similarity(movie_embeddings)

# Recommendation function using embedding similarity
def recommend(movie_title, num_recommendations=5):
    if movie_title not in movies['title'].values:
        return f"No movie found with title {movie_title}"
    index = movies[movies['title'] == movie_title].index[0]
    similarities = movie_similarity[index]
    movie_indices = np.argsort(-similarities)[1:num_recommendations+1]
    return movies.iloc[movie_indices][['title', 'year', 'popularity', 'vote_average', 'genre']]

# Example usage
print(recommend('The Godfather'))

# Save objects
pickle.dump(movies, open('movies.pkl', 'wb'))
pickle.dump(movie_similarity, open('movie_similarity.pkl', 'wb'))


Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 93ms/step - loss: 0.2373 - val_loss: 0.1156
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - loss: 0.0529 - val_loss: 0.0045
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - loss: 9.4543e-04 - val_loss: 0.0014
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 6.9970e-04 - val_loss: 0.0011
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
                               title  year  popularity  vote_average  \
495                      Shoplifters  2018    0.001320      0.804878   
76   My Hero Academia: Heroes Rising  2019    0.021465      0.902439   
25                 Life Is Beautiful  1997    0.005263      0.951220   
121                             Coco  2017    0.019902      0.878049   
251              Weathering with You  2019    0.011477      0.853659   

                                               genre  
495                                        